# COVID-19 Interactive Dashboard

In [20]:
# nbi:hide_in
import pandas as pd
from ipyleaflet import Map, GeoData, LayersControl, FullScreenControl, SplitMapControl, Choropleth, WidgetControl
from branca.colormap import linear
import json
import os
import requests
from ipywidgets import Dropdown
from bqplot import Lines, Figure, LinearScale, DateScale, Axis
import matplotlib.pyplot as plt

In [21]:
# nbi:hide_in
if not os.path.exists('limits_IT_regions.geojson'):
  url = 'https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_regions.geojson'
  r = requests.get(url)
  with open('limits_IT_regions.geojson', 'w') as f:
    f.write(r.content.decode("utf-8"))

with open('limits_IT_regions.geojson', 'r') as f:
  reg_geo_data = json.load(f)

for feat in reg_geo_data['features']:
    feat['id'] = feat['properties']['reg_istat_code_num']

In [22]:
# nbi:hide_in
if not os.path.exists('limits_IT_provinces.geojson'):
  url = 'https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_provinces.geojson'
  r = requests.get(url)
  with open('limits_IT_provinces.geojson', 'w') as f:
    f.write(r.content.decode("utf-8"))

with open('limits_IT_provinces.geojson', 'r') as f:
  prov_geo_data = json.load(f)

for feat in prov_geo_data['features']:
    feat['id'] = feat['properties']['prov_istat_code_num']

In [23]:
# nbi:hide_in
reg_geo_data['features'][:] = [feat for feat in reg_geo_data['features'] if feat['properties']['reg_name'] != 'Trentino-Alto Adige/Südtirol']

l2 = [feat for feat in prov_geo_data['features'] if feat['properties']['prov_name'] in ['Bolzano/Bozen', 'Trento']]
for item in l2:
    item['properties']['reg_name'] = item['properties']['prov_name']
    reg_geo_data['features'].append(item)

for feat in reg_geo_data['features']:
    if feat['properties']['reg_name'] == "Valle d'Aosta/Vallée d'Aoste":
        feat['properties']['reg_name'] = "Valle d'Aosta"
    elif feat['properties']['reg_name'] == 'Friuli-Venezia Giulia':
        feat['properties']['reg_name'] = 'Friuli Venezia Giulia'
    #elif feat['properties']['reg_name'] == 'Emilia-Romagna':
    #    feat['properties']['reg_name'] = 'Emilia Romagna'
    elif feat['properties']['reg_name'] == 'Bolzano/Bozen':
        feat['properties']['reg_name'] = 'P.A. Bolzano'
    elif feat['properties']['reg_name'] == 'Trento':
        feat['properties']['reg_name'] = 'P.A. Trento'

In [24]:
# nbi:hide_in
repo = 'https://raw.githubusercontent.com/pcm-dpc/COVID-19'
branch = 'master'
path_to_json = 'dati-json'
naz_json = 'dpc-covid19-ita-andamento-nazionale.json'
reg_json = 'dpc-covid19-ita-regioni.json'
prov_json = 'dpc-covid19-ita-province.json'

df_naz_raw = pd.read_json(repo + '/' + branch + '/' + path_to_json + '/' + naz_json, convert_dates=['data'])
df_naz = df_naz_raw.set_index('data')
df_reg_raw = pd.read_json(repo + '/' + branch + '/' + path_to_json + '/' + reg_json, convert_dates=['data'])
df_reg = df_reg_raw.set_index('data')
df_prov_raw = pd.read_json(repo + '/' + branch + '/' + path_to_json + '/' + prov_json, convert_dates=['data'])
df_prov = df_prov_raw.set_index('data')

In [25]:
# nbi:hide_in
df_reg_today = df_reg.loc[df_reg.index.unique()[-1],:]
df_reg_today.loc[df_reg_today.denominazione_regione == 'P.A. Bolzano', 'codice_regione'] = 21
df_reg_today.loc[df_reg_today.denominazione_regione == 'P.A. Trento', 'codice_regione'] = 22
reg_choro_data = dict(zip([int(item) for item in df_reg_today['codice_regione'].to_list()],
                          df_reg_today['totale_casi'].to_list()))

In [26]:
# nbi:hide_in
df_prov_today = df_prov.loc[df_prov.index.unique()[-1],:]

prov_choro_data = dict(zip([int(item) for item in df_prov_today['codice_provincia'].to_list()],
                          df_prov_today['totale_casi'].to_list()))

In [27]:
# nbi:hide_in
reg_layer = Choropleth(geo_data=reg_geo_data,
                       choro_data=reg_choro_data,
                       colormap=linear.YlOrRd_04,
                       border_color='black',
                       style={'fillOpacity': 0.8, 'dashArray': '5, 5'}
                      )

prov_layer = Choropleth(geo_data=prov_geo_data,
                        choro_data=prov_choro_data,
                        colormap=linear.YlOrRd_04,
                        border_color='black',
                        style={'fillOpacity': 0.8, 'dashArray': '5, 5'})

## Provincie

In [28]:
# nbi:hide_in
m1 = Map(center = (42,11), zoom = 5)
m1.add_layer(prov_layer)
#control = SplitMapControl(left_layer=reg_layer, right_layer=prov_layer)
#m.add_control(control)variazione_totale_positivi
m1.add_control(FullScreenControl())
m1

Map(center=[42, 11], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

In [29]:
# nbi:hide_in
prov_name = 'Milano'

y_data1 = df_prov[df_prov.denominazione_provincia == prov_name]['totale_casi'].values
date_data1 = df_prov[df_prov.denominazione_provincia == prov_name].index.values

In [30]:
# nbi:hide_in
date_scale1 = DateScale()
y_scale1 = LinearScale()

lines1 = Lines(x=date_data1, y=y_data1, scales={'x': date_scale1, 'y': y_scale1})

ax_x1 = Axis(label='', scale=date_scale1, num_ticks=9, tick_format='%d/%m')
ax_y1 = Axis(label='Totale casi', scale=y_scale1, orientation='vertical', side='left')

figure1 = Figure(axes=[ax_x1, ax_y1], title=prov_name, marks=[lines1], animation_duration=500,
                layout={'max_height': '250px', 'max_width': '400px'})

In [31]:
# nbi:hide_in
def update_figure1(prov_name):
    lines1.y = df_prov[df_prov.denominazione_provincia == prov_name]['totale_casi'].values

    figure1.title = prov_name
    prov_layer.choro_data = dict(zip([int(item) for item in df_prov_today['codice_provincia'].to_list()],
                                    df_prov_today['totale_casi'].to_list()))

In [32]:
# nbi:hide_in
widget_control11 = WidgetControl(widget=figure1, position='bottomright')

m1.add_control(widget_control11)

def on_click11(event, feature, **kwargs):
    global prov_name

    prov_name = feature['properties']['prov_name']
    update_figure1(prov_name)

prov_layer.on_click(on_click11)

## Regioni

In [52]:
# nbi:hide_in
m2 = Map(center = (42,11), zoom = 5)
m2.add_layer(reg_layer)
#control = SplitMapControl(left_layer=reg_layer, right_layer=prov_layer)
#m.add_control(control)
m2.add_control(FullScreenControl())
m2

Map(center=[42, 11], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

In [34]:
# nbi:hide_in
reg_name = 'Lombardia'
data_name = 'totale_casi'

y_data2 = df_reg[df_reg.denominazione_regione == reg_name][data_name].values
date_data2 = df_reg[df_reg.denominazione_regione == reg_name].index.values

In [51]:
# nbi:hide_in
date_scale2 = DateScale()
y_scale2 = LinearScale()

lines2 = Lines(x=date_data2, y=y_data2, scales={'x': date_scale2, 'y': y_scale2})

ax_x2 = Axis(label='', scale=date_scale2, num_ticks=9, tick_format='%d/%m')
ax_y2 = Axis(label=data_name.capitalize().replace('_', ' '), scale=y_scale2, orientation='vertical', side='left')

figure2 = Figure(axes=[ax_x2, ax_y2], title=reg_name, marks=[lines2], animation_duration=500,
                layout={'max_height': '250px', 'max_width': '400px'})

In [36]:
# nbi:hide_in
def update_figure2(reg_name, data_name):
    lines2.y = df_reg[df_reg.denominazione_regione == reg_name][data_name].values
    ax_y2.label = data_name.capitalize().replace('_', ' ')
    figure2.title = reg_name
    reg_layer.choro_data = dict(zip([int(item) for item in df_reg_today['codice_regione'].to_list()],
                                    df_reg_today[data_name].to_list()))

In [53]:
# nbi:hide_in
widget_control21 = WidgetControl(widget=figure2, position='bottomright')

m2.add_control(widget_control21)

def on_click21(event, feature, **kwargs):
    global reg_name

    reg_name = feature['properties']['reg_name']
    update_figure2(reg_name, data_name)

reg_layer.on_click(on_click21)

In [44]:
# nbi:hide_in
dropdown = Dropdown(
    options=['ricoverati_con_sintomi',
             'terapia_intensiva',
             'totale_ospedalizzati',
             'isolamento_domiciliare',
             'totale_positivi',
             'variazione_totale_positivi',
             'nuovi_positivi',
             'dimessi_guariti',
             'deceduti',
             'totale_casi',
             'tamponi'],
    value=data_name,
    description='Data:'
)

def on_click22(change):
    global data_name

    data_name = change['new']
    update_figure2(reg_name, data_name)

dropdown.observe(on_click22, 'value')

widget_control22 = WidgetControl(widget=dropdown, position='bottomleft')

m2.add_control(widget_control22)